In [1]:
!pip install -qq accelerate
!pip install -qq --upgrade transformers
!pip install -qq bitsandbytes
!pip install -qq wget
!pip install -qq fuzzywuzzy python-Levenshtein truecase
!pip install -qq gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ..

In [2]:
IS_COLAB = False
if 'google.colab' in str(get_ipython()):
  IS_COLAB = True

In [3]:
from transformers import logging
logging.set_verbosity(logging.ERROR)

In [4]:
import torch
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_and_dispatch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, logging
torch.cuda.empty_cache()
import os,shutil,json,random,truecase,datetime,wget,tarfile,pathlib
from fuzzywuzzy import fuzz

content_dir = os.getcwd()
if IS_COLAB:
  content_dir = pathlib.Path("/content")

In [5]:
model_name = "EleutherAI/gpt-neox-20b"

In [6]:

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

In [8]:
device_map = infer_auto_device_map(
                                  model, #using the empty model reference here saves time and RAM
                                  #change max_memory to suit your setup
                                  max_memory={0: "34GiB", "cpu": "80GiB"} if IS_COLAB else {0: "19GiB",1: "19GiB", "cpu": "90GiB"},
                                  #no_split_module_classes tells accelerate what sections of the model shouldn't be split during optimization
                                  no_split_module_classes=["GPTNeoXLayer"],
                                  #specify the doubly-shrunk weights' data type
                                  dtype=torch.int8
                              )

In [9]:
model = AutoModelForCausalLM.from_pretrained(
                                  model_name,
                                  device_map=device_map,
                                  config=config,
                                  load_in_8bit=True,
                              )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [10]:
query_in = "<context>About 1850 Lincoln compiled notes for an address to young men on the subject of the profession of law. Here again we find a refreshingly candid approach, looking without pretense at the creature man. One piece of advice which Lincoln urged upon young lawyers was that they never take their whole fee in advance. To do so would place too great a strain upon human nature, which would then lack the needful spur to industry. “When fully paid beforehand, you are more than a common mortal if you can feel the same interest in the case, as if something was still in prospect for you, as well as for your client.” As in the case of the subtreasury bill, Lincoln saw the yoking of duty and self-interest as a necessity of our nature.</context>\n\n<begin chat>\n\nQuestion: What is the capital of France?\nAnswer: The capital of France is Paris.\nQuestion: How many states are in the United States?\nAnswer: There are 50 states in the United States.\nQuestion: What is the population of Canada?\nAnswer: The population of Canada is 37.59 million people as of July 1, 2020.\nQuestion: What were Lincoln's beliefs on accepting prepayment for professional services?\nAnswer:"

In [11]:
input_ids = tokenizer(query_in, return_tensors="pt").input_ids
input_token_length=input_ids.shape[1]
input_ids = input_ids.to('cuda')
output_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=input_token_length+256,
    )
output_text = tokenizer.batch_decode(output_tokens)[0]
print(output_text)

<context>About 1850 Lincoln compiled notes for an address to young men on the subject of the profession of law. Here again we find a refreshingly candid approach, looking without pretense at the creature man. One piece of advice which Lincoln urged upon young lawyers was that they never take their whole fee in advance. To do so would place too great a strain upon human nature, which would then lack the needful spur to industry. “When fully paid beforehand, you are more than a common mortal if you can feel the same interest in the case, as if something was still in prospect for you, as well as for your client.” As in the case of the subtreasury bill, Lincoln saw the yoking of duty and self-interest as a necessity of our nature.</context>

<begin chat>

Question: What is the capital of France?
Answer: The capital of France is Paris.
Question: How many states are in the United States?
Answer: There are 50 states in the United States.
Question: What is the population of Canada?
Answer: The

In [12]:
from transformers import StoppingCriteria
#modified from the EndOfFunctionCriteria class I found somewhere around here: https://huggingface.co/transformers/v4.6.0/_modules/transformers/generation_stopping_criteria.html
class EndOfQuestionCriteria(StoppingCriteria):
  def __init__(self, start_length, eof_strings, tokenizer):
    self.start_length = start_length
    self.eof_strings = eof_strings
    self.tokenizer = tokenizer
  def __call__(self, input_ids, scores, **kwargs):
    decoded_generations = self.tokenizer.batch_decode(input_ids[:, self.start_length :])
    done = []
    for decoded_generation in decoded_generations:
      done.append(any([stop_string in decoded_generation for stop_string in self.eof_strings]))
    return all(done)

In [13]:
from transformers import StoppingCriteriaList

extra_eos = ["Human:","AI:","Question:","Answer:","</"]

input_ids = tokenizer(query_in, return_tensors="pt").input_ids
input_token_length=input_ids.shape[1]
input_ids = input_ids.to('cuda')
output_tokens = model.generate(
    input_ids,
    stopping_criteria=StoppingCriteriaList([EndOfQuestionCriteria(
            input_token_length,
            extra_eos,
            tokenizer,
        ),]),
    do_sample=True,
    temperature=0.9,
    max_length=input_token_length+256,
)
output_text = tokenizer.batch_decode(output_tokens)[0]

for eos_str in extra_eos:
  if output_text.endswith(eos_str):
    output_text = output_text[:-len(eos_str)].rstrip()
    break

print(output_text)

<context>About 1850 Lincoln compiled notes for an address to young men on the subject of the profession of law. Here again we find a refreshingly candid approach, looking without pretense at the creature man. One piece of advice which Lincoln urged upon young lawyers was that they never take their whole fee in advance. To do so would place too great a strain upon human nature, which would then lack the needful spur to industry. “When fully paid beforehand, you are more than a common mortal if you can feel the same interest in the case, as if something was still in prospect for you, as well as for your client.” As in the case of the subtreasury bill, Lincoln saw the yoking of duty and self-interest as a necessity of our nature.</context>

<begin chat>

Question: What is the capital of France?
Answer: The capital of France is Paris.
Question: How many states are in the United States?
Answer: There are 50 states in the United States.
Question: What is the population of Canada?
Answer: The

In [14]:
def neox_query(
      query_in,
      extra_eos=["Human:","AI:","Question:","Answer:","</","\n"],
      tmpt=0.95,
      max_returned_tokens=256,
      min_returned_tokens=3,
    ):
  input_ids = tokenizer(query_in, return_tensors="pt").input_ids
  input_token_length=input_ids.shape[1]
  input_ids = input_ids.to('cuda')
  output_tokens = model.generate(
    input_ids,
    stopping_criteria=StoppingCriteriaList([EndOfQuestionCriteria(
        input_token_length,
        extra_eos,
        tokenizer,
      ),]),
    do_sample=True,
    temperature=tmpt,
    max_length=input_token_length+max_returned_tokens,
    min_length=input_token_length+min_returned_tokens
  )
  output_text = tokenizer.batch_decode(output_tokens)[0]
  del input_ids
  del output_tokens
  for eos_str in extra_eos:
    if output_text.endswith(eos_str):
      output_text = output_text[:-len(eos_str)].rstrip()
      break
  return output_text 

In [15]:
def analyze_single_sentiment(string_to_analyze):
    prompt = "Text: I'm so excited for my vacation next week!\nSentiment Analysis: Positive\n\nText: I hate my job.\nSentiment Analysis: Negative\n\nText: I'm not sure what to do next. \nSentiment Analysis: Neutral\n\nText: I'm feeling optimistic about the future.\nSentiment Analysis: Positive\n\nText: I don't know what to believe anymore.\nSentiment Analysis: Neutral\n\nText: I'm so proud of my accomplishments.\nSentiment Analysis: Positive\n\nText: You should feel bad about your personal accomplishments.\nSentiment Analysis: Negative\n\nText: What the hell is wrong with your company?\nSentiment Analysis: Negative\n\nText: This is stupid.\nSentiment Analysis: Negative\n\nText: You're my hero.\nSentiment Analysis: Positive\n\nText: " + string_to_analyze + "\nSentiment Analysis:"
    sentiment_eos_stop_sequences = ["Text:","Sentiment Analysis:","\n"]
    return_val = neox_query(prompt,sentiment_eos_stop_sequences)[len(prompt):]
    for ssq in sentiment_eos_stop_sequences:
        if return_val.endswith(ssq):
            return_val = return_val[:-len(ssq)]
    return return_val.strip()

In [25]:
analyze_single_sentiment("The weather is stormy")

'Negative'

In [26]:
def average_sentiment(string_to_analyze, analyses=12):
    score = 0
    for num in range(analyses):
        value = analyze_single_sentiment(string_to_analyze)
        if "negative" in value.lower():
            score -= 1
        elif "positive" in value.lower():
            score += 1
    if score >2:
        return "positive"
    elif score < -2:
        return "negative"
    return "neutral"

In [27]:
sentiment_string = "That's why my favorite movie is The Shining. No frufu symbolism, just a good simple tale about a man maintaining a hotel."
average_sentiment(sentiment_string)

'positive'

In [28]:
preprompt = """<begin chat between human and AI>
Human: Hi!
AI: Hi! How can I help you?
Human: I need some advice on how to deal with stress.
AI: Sure, I can help with that. What kind of techniques have you tried so far?
Human: I've tried taking deep breaths and going for a walk.
AI: Those are great techniques. Have you tried any other methods?
Human: Yes, I've also tried meditating.
AI: Meditation can be a great tool for managing stress. Can you tell me more about how you meditate?
Human: Sure. I usually focus on my breath and try to clear my thoughts.
AI: That sounds like a great way to meditate. Have you found it to be helpful?
Human: Yes, it really helps me relax and refocus my energy.
AI: That's great to hear. Are there any other techniques that you'd like to try?
Human: Maybe some yoga or stretching exercises?
AI: Absolutely! Yoga and stretching are both great ways to reduce stress. Do you have any other questions?
Human: """

next_chat_input = "Tell me about chemistry."

suffix = "\nAI:"

full_prompt = preprompt + next_chat_input + suffix

reply = neox_query(full_prompt)

print(reply)

<begin chat between human and AI>
Human: Hi!
AI: Hi! How can I help you?
Human: I need some advice on how to deal with stress.
AI: Sure, I can help with that. What kind of techniques have you tried so far?
Human: I've tried taking deep breaths and going for a walk.
AI: Those are great techniques. Have you tried any other methods?
Human: Yes, I've also tried meditating.
AI: Meditation can be a great tool for managing stress. Can you tell me more about how you meditate?
Human: Sure. I usually focus on my breath and try to clear my thoughts.
AI: That sounds like a great way to meditate. Have you found it to be helpful?
Human: Yes, it really helps me relax and refocus my energy.
AI: That's great to hear. Are there any other techniques that you'd like to try?
Human: Maybe some yoga or stretching exercises?
AI: Absolutely! Yoga and stretching are both great ways to reduce stress. Do you have any other questions?
Human: Tell me about chemistry.
AI: Chemistry is the study of matter at the atom

In [32]:
next_chat_input = "What is Machine Learning"

full_prompt = reply + "\nHuman: " + next_chat_input + suffix

reply2 = neox_query(full_prompt)

print(reply2)

<begin chat between human and AI>
Human: Hi!
AI: Hi! How can I help you?
Human: I need some advice on how to deal with stress.
AI: Sure, I can help with that. What kind of techniques have you tried so far?
Human: I've tried taking deep breaths and going for a walk.
AI: Those are great techniques. Have you tried any other methods?
Human: Yes, I've also tried meditating.
AI: Meditation can be a great tool for managing stress. Can you tell me more about how you meditate?
Human: Sure. I usually focus on my breath and try to clear my thoughts.
AI: That sounds like a great way to meditate. Have you found it to be helpful?
Human: Yes, it really helps me relax and refocus my energy.
AI: That's great to hear. Are there any other techniques that you'd like to try?
Human: Maybe some yoga or stretching exercises?
AI: Absolutely! Yoga and stretching are both great ways to reduce stress. Do you have any other questions?
Human: Tell me about chemistry.
AI: Chemistry is the study of matter at the atom

In [ ]:
preprompt = """Full Text: The world is an ever-changing place, and with it comes the necessity to adapt and evolve. Technology has become the driving force behind much of the change that we see today, and it is no surprise that it is being used to shape the future of our world. From the development of autonomous vehicles to the introduction of artificial intelligence, technology is being used to make life easier and more efficient. The increasing use of technology has also had a profound impact on the way we communicate. Social media has become the primary platform for communicating with each other, and it has become a powerful tool for businesses to reach out to their customers. Technology has also enabled us to create virtual reality experiences that can provide immersive and interactive experiences for those who use them. These advancements have made the world more connected than ever before, and it has opened up new opportunities for people to collaborate and create projects. Technology has also enabled us to automate certain processes, freeing up time for us to focus on more meaningful activities. As technology continues to evolve, it will continue to shape the way we live and interact with one another. It is an exciting time to be part of the tech revolution and to witness the changes that technology is bringing to the world.
tl;dr: Technology has become the driving force behind much of the change that we see today, and it has had a profound impact on the way we communicate and interact with each other. It has enabled us to create virtual reality experiences and automate certain processes, and it will continue to shape the way we live and interact with one another.

Full Text: It is important to understand the concept of emotional intelligence, especially in the workplace. Emotional intelligence is the ability to understand and manage one’s own emotions, as well as the emotions of those around them. It is a valuable skill to have, as it helps to create a more positive and productive work environment. Being emotionally intelligent can help individuals to better identify and understand the emotions of others, which can help to improve communication and collaboration. It can also help with decision-making, problem-solving, and conflict resolution. Having a good understanding of emotional intelligence can help to create a more harmonious work atmosphere, which can help to increase productivity, morale, and overall job satisfaction.
tl;dr: Emotional intelligence is the ability to understand and manage one’s own emotions, as well as the emotions of those around them. It can help to create a more positive and productive work environment, improve communication and collaboration, and aid in decision-making and conflict resolution."""

text_for_summarization = "The success of a business is dependent on the quality of its customer service. Offering excellent customer service is the key to gaining and keeping customers, as well as encouraging customers to refer others to the business. It is important that customer service staff are well-trained and have the knowledge to handle customer inquiries and complaints in a professional manner. It is also beneficial to ensure customers have multiple contact channels to raise any queries or issues they may have, such as web chat, phone, email, and social media. Furthermore, it is important to have a system in place to respond quickly and effectively to inquiries and ensure customer satisfaction. Companies should also track customer feedback to identify any trends or issues that may require further attention. By providing a consistently high level of customer service, businesses can build a strong reputation and foster customer loyalty."

prompt = preprompt + "\n\nFull Text: " + text_for_summarization + "\ntl;dr:"

print("Text to summarize: " + text_for_summarization,"\n\nSummary: " + neox_query(prompt)[len(prompt):])

Text to summarize: The success of a business is dependent on the quality of its customer service. Offering excellent customer service is the key to gaining and keeping customers, as well as encouraging customers to refer others to the business. It is important that customer service staff are well-trained and have the knowledge to handle customer inquiries and complaints in a professional manner. It is also beneficial to ensure customers have multiple contact channels to raise any queries or issues they may have, such as web chat, phone, email, and social media. Furthermore, it is important to have a system in place to respond quickly and effectively to inquiries and ensure customer satisfaction. Companies should also track customer feedback to identify any trends or issues that may require further attention. By providing a consistently high level of customer service, businesses can build a strong reputation and foster customer loyalty. 

Summary:  By providing excellent customer servic

In [ ]:
preprompt = """Text: We leave it for the decision of each individual (after he has possessed himself of competent information as to the characteristics of each engine) to determine how far it ought to be matter of regret that such an accession has been made to the powers of human science, even if it has (which we greatly doubt) increased to a certain limited extent some already existing difficulties that had arisen in the way of completing a valuable but lesser work. We leave it for each to satisfy himself as to the wisdom of desiring the obliteration (were that now possible) of all records of the more perfect invention, in order that the comparatively limited one might be finished. The Difference Engine would doubtless fulfil all those practical objects which it was originally destined for. It would certainly calculate all the tables that are more directly necessary for the physical purposes of life, such as nautical and other computations. Those who incline to very strictly utilitarian views may perhaps feel that the peculiar powers of the Analytical Engine bear upon questions of abstract and speculative science, rather than upon those involving every-day and ordinary human interests. These persons being likely to possess but little sympathy, or possibly acquaintance, with any branches of science which they do not find to be useful (according to their definition of that word), may conceive that the undertaking of that engine, now that the other one is already in progress, would be a barren and unproductive laying out of yet more money and labour; in fact, a work of supererogation. Even in the utilitarian aspect, however, we do not doubt that very valuable practical results would be developed by the extended faculties of the Analytical Engine; some of which results we think we could now hint at, had we the space; and others, which it may not yet be possible to foresee, but which would be brought forth by the daily increasing requirements of science, and by a more intimate practical acquaintance with the powers of the engine, were it in actual existence.
Keywords: Analytical Engine, Difference Engine, utilitarian, practical objects, physical purposes, human interests, scientific, supererogation, practical results, extended faculties, increasing requirements

Text: Ulmo loves both Elves and Men, and never abandoned them, not even when they lay under the wrath of the Valar. At times he win come unseen to the shores of Middle-earth, or pass far inland up firths of the sea, and there make music upon his great horns, the Ulumúri, that are wrought of white shell; and those to whom that music comes hear it ever after in their hearts, and longing for the sea never leaves them again. But mostly Ulmo speaks to those who dwell in Middle-earth with voices that are heard only as the music of water. For all seas, lakes, rivers, fountains and springs are in his government; so that the Elves say that the spirit of Ulmo runs in all the veins of the world. Thus news comes to Ulmo, even in the deeps, of all the needs and griefs of Arda, which otherwise would be hidden from Manwë.
Keywords: Ulmo, Elves, Men, Valar, Middle-earth, Ulumúri, music, voices, seas, lakes, rivers, fountains, springs, spirit, news, needs, griefs, Arda, Manwë

Text: Smoot-Hawley Tariff Act, formally United States Tariff Act of 1930, also called Hawley-Smoot Tariff Act, U.S. legislation (June 17, 1930) that raised import duties to protect American businesses and farmers, adding considerable strain to the international economic climate of the Great Depression. The act takes its name from its chief sponsors, Senator Reed Smoot of Utah, chairman of the Senate Finance Committee, and Representative Willis Hawley of Oregon, chairman of the House Ways and Means Committee. It was the last legislation under which the U.S. Congress set actual tariff rates.
Keywords: Smoot-Hawley Tariff Act, United States Tariff Act of 1930, Hawley-Smoot Tariff Act, Reed Smoot, Willis Hawley, Senate Finance Committee, House Ways and Means Committee"""

text_to_extract_keywords_from = "But the men were hardly able to answer any questions yet, until they had learned something more about the strange creation before them. Mickey shied away, as the timid steed does at first sight of the locomotive, observing which, the boy (at a suggestion from Baldy), gave a string in his hand a twitch, whereupon the nose of the wonderful thing threw out a jet of steam with the sharp screech of the locomotive whistle. Mickey sprung a half dozen feet backward, and would have run off at full speed down the ravine, had not Ethan Hopkins caught his arm."

suffix = "\nKeywords:"

prompt = preprompt + "\n\nText: " + text_to_extract_keywords_from + suffix

print(neox_query(prompt,extra_eos = ["\n","Keywords:","Text:","</"]))

Text: We leave it for the decision of each individual (after he has possessed himself of competent information as to the characteristics of each engine) to determine how far it ought to be matter of regret that such an accession has been made to the powers of human science, even if it has (which we greatly doubt) increased to a certain limited extent some already existing difficulties that had arisen in the way of completing a valuable but lesser work. We leave it for each to satisfy himself as to the wisdom of desiring the obliteration (were that now possible) of all records of the more perfect invention, in order that the comparatively limited one might be finished. The Difference Engine would doubtless fulfil all those practical objects which it was originally destined for. It would certainly calculate all the tables that are more directly necessary for the physical purposes of life, such as nautical and other computations. Those who incline to very strictly utilitarian views may pe